# SKA-Mid conversion guide

In [2]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

XRADIO version 0.0.58 already installed.


## Download dataset

In [3]:
import toolviper

toolviper.utils.data.download(file="AA2-Mid-sim_00000.ms")

[2025-08-28 18:30:10,068]     INFO    viperlog:  Downloading from [cloudflare] .... 


 Download List         
 ────────────────────── 
  AA2-Mid-sim_00000.ms

Output()

## Convert to Processing Set

In [4]:
from xradio.measurement_set import convert_msv2_to_processing_set

ms_file = "AA2-Mid-sim_00000.ms"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "AA2-Mid-sim_00000.ps.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel_mode="none",
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2025-08-28 18:30:13,178]     INFO    viperlog:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBSERVATION_ID', 'FIELD_ID'] 
[2025-08-28 18:30:13,185]     INFO    viperlog:  Number of partitions: 1 
[2025-08-28 18:30:13,186]     INFO    viperlog:  OBSERVATION_ID [0], DDI [0], STATE [0], FIELD [0], SCAN [0] 
[2025-08-28 18:30:13,638]  WARNING    viperlog:  SOURCE table empty for (unique) source_id [0] and spectral_window_id 0. 


## Processing Set

In [5]:
ps_name = outfile

from xradio.measurement_set import open_processing_set

ps_xdt = open_processing_set(ps_store=outfile)
ps_xdt.xr_ps.summary()

,name,intents,shape,polarization,scan_name,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,AA2-Mid-sim_00000_0,[obs_0],"(95, 1891, 3, 4)","[XX, XY, YX, YY]",[0],spw_0,[_0],[Unknown],[],"[fk5, 0h00m00.00s, -45d00m00.00s]",950000000.0,9.500989e+08


In [6]:
ms_xdt = ps_xdt["AA2-Mid-sim_00000_0"]

In [7]:
ms_xdt.xr_ms.get_field_and_source_xds()

<xarray.DatasetView> Size: 16kB
Dimensions:                       (field_name: 1, sky_dir_label: 2,
                                   baseline_id: 1891, frequency: 3,
                                   polarization: 4, time: 95, uvw_label: 3)
Coordinates:
  * baseline_id                   (baseline_id) int64 15kB 0 1 2 ... 1889 1890
  * frequency                     (frequency) float64 24B 9.5e+08 ... 9.501e+08
  * polarization                  (polarization) <U2 32B 'XX' 'XY' 'YX' 'YY'
  * time                          (time) float64 760B 9.467e+08 ... 9.467e+08
  * uvw_label                     (uvw_label) <U1 12B 'u' 'v' 'w'
  * field_name                    (field_name) <U23 92B '_0'
  * sky_dir_label                 (sky_dir_label) <U3 24B 'ra' 'dec'
    source_name                   (field_name) <U7 28B 'Unknown'
Data variables:
    FIELD_PHASE_CENTER_DIRECTION  (field_name, sky_dir_label) float64 16B 0.0...
Attributes:
    type:     field_and_source

In [8]:
ms_xdt.antenna_xds

<xarray.DataTree 'antenna_xds'>
Group: /AA2-Mid-sim_00000_0/antenna_xds
    Dimensions:                 (time: 95, baseline_id: 1891, frequency: 3,
                                 polarization: 4, uvw_label: 3, antenna_name: 62,
                                 cartesian_pos_label: 3, receptor_label: 2)
    Coordinates:
      * antenna_name            (antenna_name) <U5 1kB 's0000' 's0001' ... 's0061'
      * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
        mount                   (antenna_name) <U5 1kB ...
        polarization_type       (antenna_name, receptor_label) <U1 496B ...
      * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
        station_name            (antenna_name) <U5 1kB ...
        telescope_name          (antenna_name) <U35 9kB ...
    Inherited coordinates:
      * baseline_id             (baseline_id) int64 15kB 0 1 2 3 ... 1888 1889 1890
      * frequency               (frequency) float64 24B 9.5e+08 9.5e+08 9.501e+08
      * polarization            (polarization) <U2 32B 'XX' 'XY' 'YX' 'YY'
      * time                    (time) float64 760B 9.467e+08 ... 9.467e+08
      * uvw_label               (uvw_label) <U1 12B 'u' 'v' 'w'
    Data variables:
        ANTENNA_DISH_DIAMETER   (antenna_name) float64 496B ...
        ANTENNA_POSITION        (antenna_name, cartesian_pos_label) float64 1kB ...
        ANTENNA_RECEPTOR_ANGLE  (antenna_name, receptor_label) float64 992B ...
    Attributes:
        overall_telescope_name:  OSKAR 2.8.4-dev 2023-08-22 d1b86f8c
        relocatable_antennas:    False
        type:                    antenna

In [9]:
ms_xdt.ds

<xarray.DatasetView> Size: 35MB
Dimensions:                     (time: 95, baseline_id: 1891, frequency: 3,
                                 polarization: 4, uvw_label: 3)
Coordinates:
    baseline_antenna1_name      (baseline_id) <U5 38kB ...
    baseline_antenna2_name      (baseline_id) <U5 38kB ...
  * baseline_id                 (baseline_id) int64 15kB 0 1 2 ... 1889 1890
    field_name                  (time) <U23 9kB ...
  * frequency                   (frequency) float64 24B 9.5e+08 ... 9.501e+08
  * polarization                (polarization) <U2 32B 'XX' 'XY' 'YX' 'YY'
    scan_name                   (time) <U21 8kB '0' '0' '0' '0' ... '0' '0' '0'
  * time                        (time) float64 760B 9.467e+08 ... 9.467e+08
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 1MB ...
    FLAG                        (time, baseline_id, frequency, polarization) bool 2MB ...
    TIME_CENTROID               (time, baseline_id) float64 1MB ...
    UVW                         (time, baseline_id, uvw_label) float64 4MB ...
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 17MB ...
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 9MB ...
Attributes:
    creation_date:     2025-08-28T16:30:13.280031+00:00
    creator:           {'software_name': 'xradio', 'version': '0.0.58'}
    data_groups:       {'base': {'correlated_data': 'VISIBILITY', 'date': '20...
    observation_info:  {'intents': ['obs_0'], 'observer': ['fred'], 'project'...
    processor_info:    {'sub_type': '', 'type': ''}
    schema_version:    4.0.-9988
    type:              visibility